## Generatic synthetic data using a NLME model


In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
import uuid

from vpop_calibration import *

%load_ext autoreload
%autoreload 2

In [ ]:
parameter_names = ["k_a", "k_12", "k_21", "k_el", "dose"]


def equations(t, y, *params):
    # y[0] is A_absorption, y[1] is A_central, y[2] is A_peripheral
    A_absorption, A_central, A_peripheral = y[0], y[1], y[2]
    k_a, k_12, k_21, k_el, dose = params
    dA_absorption_dt = -k_a * A_absorption
    dA_central_dt = (
        k_a * A_absorption + k_21 * A_peripheral - k_12 * A_central - k_el * A_central
    )
    dA_peripheral_dt = k_12 * A_central - k_21 * A_peripheral

    ydot = [dA_absorption_dt, dA_central_dt, dA_peripheral_dt]
    return ydot


def init_assignment(*params):
    k_a, k_12, k_21, k_el, dose = params
    return [dose, 0.0, 0.0]


variable_names = ["A0", "A1", "A2"]

pk_two_compartments_abs_model = OdeModel(
    equations, init_assignment, variable_names, parameter_names, multithreaded=False
)

In [ ]:
# Create an NLME model
# First the structural model
# We need an ODE model, initial conditions and time steps

ode_model = pk_two_compartments_abs_model
print(ode_model.variable_names)

time_span = (0, 24)
nb_steps = 20
time_steps = np.linspace(time_span[0], time_span[1], nb_steps).tolist()

# Create a protocol design, overriding one of the model's parameters
protocol_design = pd.DataFrame({"protocol_arm": ["arm-A", "arm-B"], "dose": [0.5, 2.0]})

# NLME model parameters
init_log_MI = {"k_a": 0.5}
init_log_PDU = {
    "k_12": {"mean": -1, "sd": 0.25},
    "k_el": {"mean": -1, "sd": 0.25},
}
error_model_type = "proportional"
init_res_var = [0.1, 0.2, 0.1]
covariate_map = {
    "k_12": {
        "foo": {"coef": "cov_foo_k12", "value": 0.5},
        "bar": {"coef": "cov_bar_k12", "value": 1.5},
    },
    "k_el": {},
}  # list for each PDU, which covariate influences it, and the name of the coefficient

# Define the patients data frame
nb_patients = 80
# Give them a unique id
patients_df = pd.DataFrame({"id": [f"pt-{i:d}" for i in range(nb_patients)]})
# Initialize some random properties
rng = np.random.default_rng()
# Assign a protocol arm to each patient
patients_df["protocol_arm"] = rng.binomial(1, 0.5, nb_patients)
patients_df["protocol_arm"] = patients_df["protocol_arm"].apply(
    lambda x: "arm-A" if x == 0 else "arm-B"
)
# Add the missing model descriptor as a PDK
patients_df["k_21"] = rng.normal(0.5, 0.01, nb_patients)
# Add covariates
patients_df["foo"] = rng.normal(1, 0.1, nb_patients)
patients_df["bar"] = rng.normal(2, 0.5, nb_patients)
display(patients_df)

In [ ]:
obs_df = simulate_dataset_from_omega(
    ode_model,
    protocol_design,
    time_steps,
    init_log_MI,
    init_log_PDU,
    error_model_type,
    init_res_var,
    covariate_map,
    patients_df,
)

display(obs_df.head())

In [ ]:
from plotnine import *

(
    ggplot(obs_df, aes(x="time", y="value", color="id"))
    + geom_line()
    + facet_grid(rows="protocol_arm", cols="output_name")
    + theme(legend_position="none")
)